In [2]:
from supabase import create_client
import pandas as pd
import utils.env as env

url = env.SUPABASE_URL
key = env.SUPABASE_KEY
supabase = create_client(url, key)


def _get_all_album_statistics(artist_uri):
    rows_list = []

    album_response = supabase.table("albums")\
        .select("album_name, metrics(song_name, popularity, danceability, energy, valence)")\
        .like('artist_uris', f'%{artist_uri}%')\
        .execute()
    all_album_data = album_response.data
    # returns list of dictionaries. each entry here is album. album contains {album_name:"name", metrics:[{}, {}, {}... ] } 
    # enumerate through album LIST, each album is dict
    for idx, album in enumerate(all_album_data):
        album_name = {"album_name": album.get('album_name')}
        print(idx, album_name)
        # # [ {}, {}, {} ]
        metrics = album.get('metrics')
        # # enumerate through song LIST, each song is dict of metrics
        for song_idx, song_metrics in enumerate(metrics):
            new_row = album_name.copy()
            new_row.update(song_metrics)
            #print(new_row)
            rows_list.append(new_row)
    # reorder for viewing
    return_df = pd.DataFrame(rows_list)
    return return_df[["song_name", "album_name", "popularity", "danceability", "energy", "valence"]]


resp = _get_all_album_statistics("spotify:artist:699OTQXzgjhIYAHMy9RyPD")
# returns list of dictionaries. each entry here is album. album contains {album_name:"name", metrics:[{}, {}, {}... ] } 
print(resp)




2024-01-23 15:27:42,644:INFO - HTTP Request: GET https://snuqfngjrznygycntzst.supabase.co/rest/v1/albums?select=album_name%2C%20metrics%28song_name%2C%20popularity%2C%20danceability%2C%20energy%2C%20valence%29&artist_uris=like.%25spotify%3Aartist%3A699OTQXzgjhIYAHMy9RyPD%25 "HTTP/1.1 200 OK"


0 {'album_name': 'Whole Lotta Red'}
1 {'album_name': 'Die Lit'}
2 {'album_name': 'Playboi Carti'}
                                       song_name       album_name  popularity  \
0                                  Rockstar Made  Whole Lotta Red          73   
1                   Go2DaMoon (feat. Kanye West)  Whole Lotta Red          72   
2                                 Stop Breathing  Whole Lotta Red          75   
3                                          Beno!  Whole Lotta Red          65   
4                                JumpOutTheHouse  Whole Lotta Red          66   
5                 M3tamorphosis (feat. Kid Cudi)  Whole Lotta Red          66   
6                                         Slay3r  Whole Lotta Red          65   
7                                       No Sl33p  Whole Lotta Red          64   
8                                       New Tank  Whole Lotta Red          78   
9                          Teen X (feat. Future)  Whole Lotta Red          64   
10         

In [4]:
resp.head(5)

,song_name,album_name,popularity,danceability,energy,valence
0,Rockstar Made,Whole Lotta Red,73,0.773,0.612,0.381
1,Go2DaMoon (feat. Kanye West),Whole Lotta Red,72,0.693,0.434,0.542
2,Stop Breathing,Whole Lotta Red,75,0.856,0.570,0.155
3,Beno!,Whole Lotta Red,65,0.923,0.562,0.499
4,JumpOutTheHouse,Whole Lotta Red,66,0.870,0.461,0.485


In [21]:
averages = resp.groupby("album_name")[["popularity", "danceability", "energy", "valence"]].mean().round(2)
averages["album name"] = averages.index

In [22]:
averages

,popularity,danceability,energy,valence,album name
album_name,,,,,
Die Lit,68.00,0.76,0.66,0.41,Die Lit
Playboi Carti,66.13,0.79,0.60,0.46,Playboi Carti
Whole Lotta Red,68.92,0.75,0.66,0.34,Whole Lotta Red
